In [1]:
# Partie 2 ex3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Chargement des données Boston depuis la source originale
data_url = "http://lib.stat.cmu.edu/datasets/boston"
try:
    # Lecture directe de l'URL
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]

    # Créer un DataFrame avec les noms de colonnes appropriés
    column_names = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat']
    Boston = pd.DataFrame(data, columns=column_names)
    Boston['medv'] = target
except:
    # Alternative si l'URL ne fonctionne pas
    url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
    Boston = pd.read_csv(url)
    # Assurer que les noms de colonnes sont en minuscules comme en R
    Boston.columns = [col.lower() for col in Boston.columns]

# Afficher les 6 premières lignes
print(Boston.head(6))

# Obtenir un résumé des données
print(Boston.describe())

# Affiche la structure des données
print(Boston.info())

# Séparer en training (400 premières lignes) et test (reste)
train_set = Boston.iloc[:400, :]
test_set = Boston.iloc[400:, :]

# Analyse de corrélation
print(f"Corrélation entre medv et age: {Boston['medv'].corr(Boston['age'])}")

# Visualisation de la relation entre Age et Medv
plt.figure(figsize=(10, 6))
plt.scatter(Boston['age'], Boston['medv'], color='blue', marker='o')
plt.xlabel('Age')
plt.ylabel('Valeur Médiane')
plt.title('Relation entre Age et Medv')

# Modèle de régression linéaire pour age
model = sm.OLS(Boston['medv'], sm.add_constant(Boston['age'])).fit()
x_range = np.linspace(Boston['age'].min(), Boston['age'].max(), 100)
y_pred = model.params[0] + model.params[1] * x_range
plt.plot(x_range, y_pred, color='red', linewidth=2)
plt.show()

# Visualisation de la relation entre lstat et medv
plt.figure(figsize=(10, 6))
plt.scatter(Boston['lstat'], Boston['medv'], color='blue', marker='o')
plt.xlabel('LSTAT')
plt.ylabel('Valeur Médiane')
plt.title('Relation entre LSTAT et MEDV')

# Modèle de régression linéaire pour lstat
model_lstat = sm.OLS(Boston['medv'], sm.add_constant(Boston['lstat'])).fit()
x_range = np.linspace(Boston['lstat'].min(), Boston['lstat'].max(), 100)
y_pred = model_lstat.params[0] + model_lstat.params[1] * x_range
plt.plot(x_range, y_pred, color='red', linewidth=2)
plt.show()

# Prédictions sur l'ensemble de test
X_test = sm.add_constant(test_set['age'])
predictions = model.predict(X_test)
print("Prédictions:", predictions.head())

# Histogramme de MEDV
plt.figure(figsize=(10, 6))
plt.hist(Boston['medv'], bins=30, color='lightblue')
plt.title('Distribution de MEDV')
plt.show()

# QQ-plot pour vérifier la normalité
plt.figure(figsize=(10, 6))
stats.probplot(Boston['medv'], plot=plt)
plt.title('QQ Plot de MEDV')
plt.show()

# Résumé du modèle de régression
print(model.summary())

# Matrice de dispersion (équivalent de pairs())
sns.pairplot(Boston)
plt.show()

# Visualisation des relations entre 3 variables spécifiques
sns.pairplot(Boston.iloc[:, [0, 2, 6]])  # 1ère, 3ème et 7ème variable
plt.show()

# Régression linéaire multiple
formula = 'medv ~ lstat + age'
model_multi = ols(formula, data=train_set).fit()
print(model_multi.summary())

# Tester un modèle avec transformation logarithmique
train_set_log = train_set.copy()
train_set_log['log_lstat'] = np.log(train_set_log['lstat'])
formula_log = 'medv ~ log_lstat + age'
model_log = ols(formula_log, data=train_set_log).fit()
print(model_log.summary())

# Matrice de corrélation
correlation_matrix = Boston.corr()
print(correlation_matrix)

# Modèle final avec variables significatives
formula_final = 'medv ~ crim + rad + black + lstat'
model_final = ols(formula_final, data=train_set).fit()
print(model_final.summary())

# Modèle avec interaction
formula_interaction = 'medv ~ lstat * age'
model_interaction = ols(formula_interaction, data=train_set).fit()
print(model_interaction.summary())

# Teste des modèles de polynômes jusqu'au degré 7
for d in range(2, 8):
    # Création de caractéristiques polynomiales pour reproduire poly() de R
    poly_features = []
    for i in range(1, d+1):
        poly_features.append(np.power(train_set['lstat'], i))

    poly_df = pd.DataFrame(np.column_stack(poly_features))
    # Ajouter une constante pour l'intercept
    poly_df = sm.add_constant(poly_df)

    # Ajustement du modèle
    model_poly = sm.OLS(train_set['medv'], poly_df).fit()
    print(f"\nRésultats pour le polynôme de degré {d}:")
    print(model_poly.summary())

Output hidden; open in https://colab.research.google.com to view.